In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [2]:
scenarios = [
    #{"model": "image", "pathway": "SSP2-Base", "year": 2020},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2022},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2025},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2030},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2040},
    {"model": "image", "pathway": "SSP2-RCP26", "year": 2050},
]

sps = Package("../datapackage.json")

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[sps,],
        use_multiprocessing=False
)

premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [3]:
ndb.update()

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: metals:  55%|████████▋       | 6/11 [01:51<01:50, 22.08s/it]

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Updating: external:  91%|███████████▊ | 10/11 [04:14<00:23, 23.81s/it]

No suppliers found for market for heavy fuel oil (SPS) in CH. No market created. This may cause linking issue.


Updating: external: 100%|█████████████| 11/11 [04:19<00:00, 23.58s/it]

Done!



In [4]:
ndb.write_db_to_brightway("stem test 3")

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Title: Writing activities to SQLite3 database:
  Started: 03/27/2024 17:47:31
  Finished: 03/27/2024 17:47:51
  Total time elapsed: 00:00:19
  CPU %: 92.80
  Memory %: 5.98
Created database: stem test 3
Generate scenario report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/sweet_sure-2050-switzerland/dev.


In [5]:
import numpy as np
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if np.isnan(e["amount"]):
            print(e)

In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")
sps = Package("../datapackage.json")
ndb = PathwaysDataPackage(
    scenarios=[
        {"model": "image", "pathway": "SSP2-RCP26"},
    ],
    #years=[2020, ],
    years=[2020, 2022, 2025, 2030, 2035, 2040, 2045, 2050],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    external_scenarios=[sps,],
    use_multiprocessing=True
)

premise v.(2, 0, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [2]:
ndb.create_datapackage(
    name="image-SSP2-stem-SPS1",
    contributors=[
        {"name": "Romain",
        "email": "r_s at me.com",}
    ],
    export_uncertainty=True
)

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: heat:  64%|██████████▊      | 7/11 [14:14<11:32, 173.04s/it]

Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal spodumene not found in alternative names. Skipping transport.
Metal rare earth carbonate concentrate not found in alternative names. Skipping transport.
Metal rare earth carbonat

Updating: external:  91%|██████████▉ | 10/11 [18:48<01:54, 114.46s/it]

No suppliers found for market for biodiesel (SPS) in CH. No market created. This may cause linking issue.
No suppliers found for market for biodiesel (SPS) in CH. No market created. This may cause linking issue.


Updating: external: 100%|████████████| 11/11 [19:40<00:00, 107.31s/it]


Done!

Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.02 seconds
Remove uncertainty data.
Write new database(s) to matrix.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Matrices saved in /Users/romain/Github/sweet_sure-2050-switzerla

In [1]:
from pathways import Pathways
p = Pathways(datapackage="image-SSP2-stem-SPS1/datapackage.json")

Solver: scikits.umfpack
Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/41" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/41/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile


In [2]:
import numpy as np
p.calculate(
    methods=[
        #'IPCC 2021 - climate change - GWP 100a, incl. H',
        'IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2',
        #'RELICS - metals extraction - Lithium',
        #'RELICS - metals extraction - Zinc',
        #'RELICS - metals extraction - Praseodymium',
        #'RELICS - metals extraction - Cobalt',

    ],
    regions=["CH",],
    scenarios=[
        "SSP2-RCP26",
        #"SSP2-RCP19",
    ],
    variables=[
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CCS)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CCS)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ],
    years=[
        2020,
        2022,
        2030,
        2040,
        2050
    ],
    characterization=True,
    #data_type=np.float32,
    multiprocessing=False,
)

TypeError: Cannot cast array data from dtype([('row', '<i4'), ('col', '<i4')]) to dtype('int64') according to the rule 'unsafe'

In [5]:
p.lca_results.sum()

<xarray.DataArray ()> Size: 8B
array(2.03569216e+10)

In [6]:
p.lca_results = p.lca_results.sum(dim="act_category")
arr = p.lca_results.interp(
    year=range(p.lca_results.coords["year"].values.min(), p.lca_results.coords["year"].values.max() + 1)
)
df = arr.to_dataframe("value")
df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
print(len(df))
df=df.reset_index()
print(df["year"].unique())

221864
[2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2020 2021 2022 2023
 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037
 2038 2039 2040]


In [7]:
from premise.geomap import Geomap
geo = Geomap("image")
d = {
    loc: geo.ecoinvent_to_iam_location(loc)
    for loc in df["location"].unique()
}
df["region from"] = df["location"].map(d)
df = df.drop('location', axis=1)
df = df.groupby([
    'variable', 'year', 'region', 'model', 'scenario', 'impact_category', 'region from'
]).sum().reset_index()
print(len(df))

19899


In [8]:
df["impact_category"] = df["impact_category"].str.replace("RELICS - metals extraction - ", "")
df["impact_category"] = df["impact_category"].str.replace("EF v3.1 - climate change - global warming potential (GWP100)", "climate change")

In [9]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')

In [12]:
from stats_arrays import UncertaintyBase

In [14]:
UncertaintyBase.from_tuples?

Signature: UncertaintyBase.from_tuples(*data)
Docstring:
Construct a :ref:`hpa` from parameter tuples.

The order of the parameters is:

#. ``loc``
#. ``scale``
#. ``shape``
#. ``minimum``
#. ``maximum``
#. ``negative``
#. ``uncertainty_type``

Each input tuple must have a length of exactly 7. For more flexibility, use ``from_dicts``.

Example:

.. code-block:: python

    >>> from stats_arrays import UncertaintyBase
    >>> import numpy as np
    >>> UncertaintyBase.from_tuples(
    ...     (2, 3, np.NaN, np.NaN, np.NaN, False, 3),
    ...     (5, np.NaN, np.NaN, 3, 10, False, 5)
    ...     )
    array([(2.0, 3.0, nan, nan, nan, False, 3),
           (5.0, nan, nan, 3.0, 10.0, False, 5)],
           dtype=[('loc', '<f8'), ('scale', '<f8'), ('shape', '<f8'),
                  ('minimum', '<f8'), ('maximum', '<f8'), ('negative', '?'),
                  ('uncertainty_type', 'u1')])

Args:
    One of more tuples of length 7.

Returns:
    A :ref:`hpa`

        
File:      ~/anaconda3/env